In [1]:
from imp import reload
import glob
import os

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
import utils; reload(utils)
images, labels = utils.load_data()

vehicle = 8792 samples
non-vehicle = 8968 samples


In [3]:
import utils; reload(utils)
%time hog_features = utils.get_hog_features(images)

CPU times: user 1min 6s, sys: 1.78 s, total: 1min 8s
Wall time: 1min 12s


In [4]:
from sklearn.utils import shuffle
X, y = shuffle(hog_features, labels, random_state=0)

In [6]:
# train, test split
from sklearn.model_selection import train_test_split

X_train_origin, X_test, y_train_origin, y_test = \
    train_test_split(X, y, random_state=0, stratify=y)

In [8]:
# SVM

import time

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# DataFrame to record cross validation result
df_cv = pd.DataFrame()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(skf.split(X_train_origin, y_train_origin)):
    t0_fold = time.time()

    X_train, y_train = X_train_origin[train_index], y_train_origin[train_index]
    X_val, y_val = X_train_origin[test_index], y_train_origin[test_index]

    clf = SVC()
    t0_fit = time.time()
    clf.fit(X_train, y_train)
    print('Fold {} in {}'.format(i + 1, time.time() - t0_fit))

    y_pred_train = clf.predict(X_train)
    y_pred_val = clf.predict(X_val)

    df_cv.loc[i, 'acc'] = accuracy_score(y_train, y_pred_train)
    df_cv.loc[i, 'val_acc'] = accuracy_score(y_val, y_pred_val)
    
    print('Fold {} in {}'.format(i + 1, time.time() - t0_fold))

print(df_cv)

Fold 1 in 624.5036978721619
Fold 1 in 1410.033364057541
Fold 2 in 632.2436780929565
Fold 2 in 1387.6230890750885
Fold 3 in 624.837021112442
Fold 3 in 1418.9804599285126
Fold 4 in 617.5587420463562
Fold 4 in 1453.616994857788
Fold 5 in 648.9987089633942
Fold 5 in 1479.9915750026703
        acc   val_acc
0  0.874894  0.873171
1  0.873592  0.868243
2  0.873874  0.879129
3  0.875000  0.870120
4  0.874073  0.876455


In [9]:
# Model persistence
from sklearn.externals import joblib
joblib.dump(clf, 'svm.pkl')

['svm.pkl']

In [10]:
# Test accuracy
t0 = time.time()
y_pred_test = clf.predict(X_test)
print('pred time = {} seconds'.format(time.time() - t0))
print(accuracy_score(y_test, y_pred_test))
print('test acc = ', clf.score(X_test, y_test))

pred time = 262.16670203208923 seconds
0.871846846847
test acc =  0.871846846847


### Linear SVM

In [13]:
# Linear SVM
import time

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC


# DataFrame to record cross validation result
df_cv_linear_svc = pd.DataFrame()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(skf.split(X_train_origin, y_train_origin)):
    print('Start Fold {}'.format(i + 1))
    t0_fold = time.time()

    X_train, y_train = X_train_origin[train_index], y_train_origin[train_index]
    X_val, y_val = X_train_origin[test_index], y_train_origin[test_index]

    clf = LinearSVC()
    t0_fit = time.time()
    clf.fit(X_train, y_train)
    print('Fit in {}'.format(time.time() - t0_fit))

    df_cv_linear_svc.loc[i, 'acc'] = clf.score(X_train, y_train)
    df_cv_linear_svc.loc[i, 'val_acc'] = clf.score(X_val, y_val)
    
    print('Fold {} in {}'.format(i + 1, time.time() - t0_fold))

print(df_cv_linear_svc)

# Model persistence
from sklearn.externals import joblib
joblib.dump(clf, 'linear_svm.pkl')

Start Fold 1
Fit in 13.026461839675903
Fold 1 in 13.512642860412598
Start Fold 2
Fit in 12.42301893234253
Fold 2 in 12.857972145080566
Start Fold 3
Fit in 12.231029987335205
Fold 3 in 12.686918020248413
Start Fold 4
Fit in 15.79684591293335
Fold 4 in 16.356952905654907
Start Fold 5
Fit in 15.343703985214233
Fold 5 in 15.86599087715149
        acc   val_acc
0  0.996997  0.962852
1  0.997279  0.971847
2  0.996622  0.971847
3  0.997185  0.973348
4  0.997466  0.964326


['linear_svm.pkl']

In [14]:
# Test accuracy
%time print('test acc = ', clf.score(X_test, y_test))

test acc =  0.968918918919
CPU times: user 60.9 ms, sys: 197 ms, total: 258 ms
Wall time: 407 ms
